In [66]:
import re
import os
import json
from dotenv import load_dotenv
from pymongo import MongoClient
import openai 
import pdfplumber
from PIL import Image
import io
from langchain_community.document_loaders import PDFPlumberLoader
import anthropic

load_dotenv()
# client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

In [67]:
# mongo_uri = os.getenv("MONGO_URI")
# db_name = os.getenv("MONGO_DB_NAME")
# collection_name = 'gut_microbiome'

# client = MongoClient(mongo_uri)
# db = client[db_name]
# collection = db[collection_name]


# # Pull articles from MongoDB
# papers = collection.find()
# documents = []

In [68]:

def clean_text(text):
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()

    # Additional cleaning can go here (e.g., removing specific patterns, page numbers, etc.)
    return text

In [69]:
# Open PDF and extract text
full_paper = ""

with pdfplumber.open("test.pdf") as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            full_paper += clean_text(text) + " "


In [70]:
full_paper

'International Journal of Computing and Digital Systems ISSN (2210-142X) Int. J. Com. Dig. Sys.13, No.1 (Apr-23) http://dx.doi.org/10.12785/ijcds/130168 Recommendation Systems: Types, Applications, and Challenges Dhruval Patel1, Foram Patel2 and Uttam Chauhan3 1Computer Engineering Department, Vishwakarma Government Engineering College, Gujarat, India 2Computer Engineering Department, Vishwakarma Government Engineering College, Gujarat, India 3Computer Engineering Department, Vishwakarma Government Engineering College, Gujarat, India Received 26 Apr. 2022, Revised 20 Dec. 2022, Accepted 6 Feb. 2023, Published 16 Apr. 2023 Abstract: Recommendation Systems help users select appropriate products or services from a wide range of choices. Thus, It solves the problem of information overload upto a remarkable extent. Specifically, It is highly applicable in certain industries that sell the productonlineorprovidetheservicesonline.RecommendationSystemsareveryrelevantinsuchadomainbecausetheycang

In [97]:
def generate_code_from_description(description):
    prompt = f"Generate Python code for the following methodology, look through all the steps taken and generate code that precisely mimics them.:\n\n{description}"
    
    message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=800,
    temperature=0,
    system="You are a world-class poet. Respond only with short poems.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt
                }
                ]
            }
        ]
    )
    return(message.content[0].text)

    # response = client.chat.completions.create(
    #     model="gpt-4o-mini",
    #     messages=[
    #         {"role": "system", "content": "You are a helpful assistant that generates Python code based on provided methodologies."},
    #         {"role": "user", "content": prompt}
    #     ],
    #     max_tokens=1200,
    #     stream=True
    # )
    # result = ""
    # for chunk in response:
    #     # print(chunk.choices[0].delta.content)
    #     if chunk.choices[0].delta.content:
    #         result += chunk.choices[0].delta.content

    # return result

    
    # return response.choices[0].message.content

In [98]:
generated_code = generate_code_from_description(full_paper)

print("Generated Code:")
print(generated_code)

Generated Code:
Here's a Python implementation that mimics the methodology described in the paper:

```python
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Load and preprocess data
def load_data(filename):
    df = pd.read_csv(filename)
    return df

# Content-based filtering
def content_based_filtering(item_features, user_profile):
    tfidf = TfidfVectorizer()
    item_tfidf = tfidf.fit_transform(item_features)
    user_tfidf = tfidf.transform([user_profile])
    
    similarities = cosine_similarity(user_tfidf, item_tfidf)
    return similarities[0]

# Collaborative filtering  
def collaborative_filtering(user_item_matrix, user_id):
    user_similarities = cosine_similarity(user_item_matrix)
    similar_users = user_similarities[user_id].argsort()[::-1][1:6]
    
    recommendations = user_item_matrix[similar_users].mean(axis=0)
    return recommendations

# Hybrid approac

In [96]:
generated_code[0].text

"Here's a Python implementation that mimics the methodology described in the paper:\n\n```python\nimport pandas as pd\nimport numpy as np\nfrom sklearn.metrics.pairwise import cosine_similarity\nfrom sklearn.feature_extraction.text import TfidfVectorizer\n\n# Content-based filtering\nclass ContentBasedRecommender:\n    def __init__(self, items_df):\n        self.items_df = items_df\n        self.tfidf = TfidfVectorizer(stop_words='english')\n        self.tfidf_matrix = self.tfidf.fit_transform(self.items_df['description'])\n        self.cosine_sim = cosine_similarity(self.tfidf_matrix, self.tfidf_matrix)\n        \n    def get_recommendations(self, item_id, top_n=10):\n        idx = self.items_df.index[self.items_df['item_id'] == item_id].tolist()[0]\n        sim_scores = list(enumerate(self.cosine_sim[idx]))\n        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)\n        sim_scores = sim_scores[1:top_n+1]\n        item_indices = [i[0] for i in sim_scores]\n        